<a href="https://www.kaggle.com/code/erfanramezani/diabetes-preprocessing-dnn?scriptVersionId=142898245" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# First import required libraries

numpy and pandas ( organize dataset and arrays )

sklearn ( preproccesing and evaluate model )

tesnorflow ( to build deep neural network Model )

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler , Normalizer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from tensorflow import keras


# read dataset diabetes.csv from kaggle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# import data 
dataset = pd.read_csv('../input/diabetes/diabetes.csv')
dataset.head()

# search for Nah or null data in our dataset

In [ ]:
dataset.info()

# Advance Preprocessing procedure

Since you might need to scale your own test data , or you might **save the preprocessor** to use model in another file you need to define a preprocessor object to do it automatically

In [ ]:
X = dataset.copy()
y = X.pop('Outcome')

## ------ preprocessing ------ ##

preprocessor = make_column_transformer(
    (StandardScaler(),
     make_column_selector(dtype_include=np.number)),
)

preprocessor.fit(X)
X = preprocessor.transform(X)

## ------ preprocessing ------ ##

# Compare dataset vs scaled version of it

In [ ]:
dataset.head()

In [ ]:
scaled_dataset = pd.DataFrame(X,columns=dataset.columns[:-1])
scaled_dataset.head()

# Visualization

In [ ]:
dataset.plot()

In [ ]:
scaled_dataset.plot()

> **obviously you can see how Standardization data affects on data range due our Learning  Algorithms perform better on recognizing data relationship**

# split data to train and test

In [ ]:
train_x, test_x ,train_y , test_y = train_test_split(
    X,y, test_size=0.33, random_state=1)

# DNN model definition


After you build your model you may be noticed **overfitting** 
to prevent overfitting we use **regularize Term** which Andrew Ng uses on coursera Deep Learning course

also you can try **Dropout** to see how it works 


In [ ]:
model = keras.Sequential([
 
    keras.layers.Input((8,)),
#     keras.layers.Dropout(0.3),
    keras.layers.Dense(150 ,activation='relu', kernel_regularizer = keras.regularizers.L2()),
#     keras.layers.Dropout(0.3),
    keras.layers.Dense(75 ,activation='relu', kernel_regularizer = keras.regularizers.L2(),),
    keras.layers.Dense(1 ,activation='sigmoid'),
  
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss= keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy'],
    )

history = model.fit(
    x=train_x,
    y=train_y,
    validation_data=(test_x,test_y),
    batch_size=28,
    epochs=30
)

# analyze our model

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss','val_loss']].plot();
history_df.loc[:, ['accuracy','val_accuracy']].plot();

# evaluations 

In [ ]:
model.evaluate(test_x,test_y)

In [ ]:
y__predict=model.predict(test_x)
y__real = test_y

y__predict = [ 1 if y>=0.5 else 0 for y in y__predict] #usualy decide to split on 0.5 

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(y__predict, y__real)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=cm)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(y__predict, y__real))
print('Report : ')
print(classification_report(y__predict, y__real))

# hopefully it worked well

but if you are beginner you should change and practice with some parameters to see how your model perform on test data